
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
 !pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
Receiving objects: 100% (53/53), 32.99 KiB | 3.30 MiB/s, done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Resolving deltas: 100% (31/31), done.


In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y4", label="solidos")

/tmp/ipykernel_200468/178310800.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-29 01:37:42.467369: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 01:37:42.584594: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 01:37:42.584682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 01:37:42.586146: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 01:37:42.596273: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 01:37:42.600224: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[30, 15], [45, 30], [60, 30]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[30, 15], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  2.647322520000671
Stopped at epoch:  0
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.04690508699968632
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.0493114479995711
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.04674823799996375
Stopped at epoch:  0
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.049583531000280345
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.05976067099982174
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_0] | 7 +++++

2024-02-29 01:40:01.579759: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  8.50331357899995
Stopped at epoch:  112
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_1] | 1 ++++++++++++++++++
Elapsed time:  0.5258451889994831
Stopped at epoch:  6
1/1 [==============================] - 0s 27ms/step
+++++++++++ [2_1] | 2 ++++++++++++++++++
Elapsed time:  0.5651038089999929
Stopped at epoch:  7
1/1 [==============================] - 0s 34ms/step
+++++++++++ [2_1] | 3 ++++++++++++++++++
Elapsed time:  0.4729764600006092
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_1] | 4 ++++++++++++++++++
Elapsed time:  0.6942982199998369
Stopped at epoch:  8
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_1] | 5 ++++++++++++++++++
Elapsed time:  0.5334170420001101
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_1] | 6 ++++++++++++++++++
Elapsed time:  0.45614690000002156
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
++++

2024-02-29 01:40:30.322999: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.5116544439997597
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [2_2] | 1 ++++++++++++++++++
Elapsed time:  0.6106444970000666
Stopped at epoch:  8
1/1 [==============================] - 0s 25ms/step
+++++++++++ [2_2] | 2 ++++++++++++++++++
Elapsed time:  0.6000710920006895
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [2_2] | 3 ++++++++++++++++++
Elapsed time:  0.5778260860006412
Stopped at epoch:  7
1/1 [==============================] - 0s 22ms/step
+++++++++++ [2_2] | 4 ++++++++++++++++++
Elapsed time:  0.49760438600060297
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_2] | 5 ++++++++++++++++++
Elapsed time:  0.5372818229998302
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [2_2] | 6 ++++++++++++++++++
Elapsed time:  0.4646958969997286
Stopped at epoch:  6
1/1 [==============================] - 0s 31ms/step
++++

...

1/1 [==============================] - 0s 19ms/step
+++++++++++ [10_3] | 15 ++++++++++++++++++
Elapsed time:  0.9636897520003913
Stopped at epoch:  11
1/1 [==============================] - 0s 24ms/step
+++++++++++ [10_3] | 16 ++++++++++++++++++
Elapsed time:  1.4445941570002105
Stopped at epoch:  18
1/1 [==============================] - 0s 15ms/step
+++++++++++ [10_3] | 17 ++++++++++++++++++
Elapsed time:  0.9589155579997168
Stopped at epoch:  12
1/1 [==============================] - 0s 23ms/step
+++++++++++ [10_3] | 18 ++++++++++++++++++
Elapsed time:  1.2887686139993093
Stopped at epoch:  18
1/1 [==============================] - 0s 27ms/step
+++++++++++ [10_3] | 19 ++++++++++++++++++
Elapsed time:  0.8480653010001333
Stopped at epoch:  10
1/1 [==============================] - 0s 23ms/step
+++++++++++ [10_3] | 20 ++++++++++++++++++
Elapsed time:  1.8632077310003297
Stopped at epoch:  27
1/1 [==============================] - 0s 25ms/step
+++++++++++ [10_3] | 21 ++++++++++++++++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_10_8_0,0.800239,0.330884,0.315208,0.324310,24.091217,54.115070,46.640339,50.597557
model_10_8_1,0.800253,0.328517,0.306641,0.319288,24.089504,54.306515,47.223846,50.973637
model_10_8_2,0.805826,0.347775,0.327995,0.339445,23.417431,52.749016,45.769459,49.464256
model_10_8_3,0.806976,0.361581,0.312241,0.340593,23.278790,51.632408,46.842438,49.378235
model_10_8_4,0.808250,0.365507,0.307969,0.341008,23.125059,51.314922,47.133419,49.347214
model_10_8_5,0.809930,0.379366,0.279071,0.336564,22.922495,50.194046,49.101597,49.679970
model_10_8_6,0.810721,0.380884,0.275182,0.335768,22.827105,50.071270,49.366459,49.739594
model_10_8_7,0.811197,0.398425,0.253394,0.336471,22.769669,48.652634,50.850445,49.686947


DataFrame salvo em ../results/metrics_10_8
+++++++++++ [10_9] | 1 ++++++++++++++++++
Elapsed time:  0.445570745000623
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [10_9] | 2 ++++++++++++++++++
Elapsed time:  0.4996344719993431
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [10_9] | 3 ++++++++++++++++++
Elapsed time:  0.5371189339994089
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [10_9] | 4 ++++++++++++++++++
Elapsed time:  0.5150641580003139
Stopped at epoch:  6
1/1 [==============================] - 0s 28ms/step
+++++++++++ [10_9] | 5 ++++++++++++++++++
Elapsed time:  0.5179130080005052
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [10_9] | 6 ++++++++++++++++++
Elapsed time:  0.5735946089998833
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [10_9] | 7 ++++++++++++++++++
Elapsed time:  0.43002

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_10_9_0,0.801296,0.79826,-0.389878,0.596862,23.963787,29.610874,46.895454,37.744537


DataFrame salvo em ../results/metrics_10_9
Testando combinacao11: Hidden Size=[60, 30], regularizer=0.1, learning_rate=0.02
+++++++++++ [11_0] | 1 ++++++++++++++++++
Elapsed time:  2.011027810000087
Stopped at epoch:  18
1/1 [==============================] - 0s 21ms/step
+++++++++++ [11_0] | 2 ++++++++++++++++++
Elapsed time:  1.6506110219997936
Stopped at epoch:  20
1/1 [==============================] - 0s 25ms/step
+++++++++++ [11_0] | 3 ++++++++++++++++++
Elapsed time:  0.5323526210004275
Stopped at epoch:  7
1/1 [==============================] - 0s 26ms/step
+++++++++++ [11_0] | 4 ++++++++++++++++++
Elapsed time:  0.7255702709999241
Stopped at epoch:  9
1/1 [==============================] - 0s 19ms/step
+++++++++++ [11_0] | 5 ++++++++++++++++++
Elapsed time:  0.7000458539996544
Stopped at epoch:  8
1/1 [==============================] - 0s 21ms/step
+++++++++++ [11_0] | 6 ++++++++++++++++++
Elapsed time:  0.5842052830003013
Stopped at epoch:  6
1/1 [============================